In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill
import submatcher as sm

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [3]:
allr = rr.GetRecipies(inp)

#add self.get() to __init__ and bottom 4 lines are replaced - just saving an API call for now
# dbfile = open('resp.pkl', 'rb') 
# dbfile.seek(0)
# allr.response =  dill.load(dbfile)

# allr.process()

In [4]:
allr.to_json()

{'Cabrales Cheese Souffles with Endive and Asian Pear Salad': {'attrs': {'vegetarian': False,
   'vegan': False,
   'glutenFree': False,
   'dairyFree': False},
  'cuisines': array(['Indian', 'Asian'], dtype='<U6'),
  'id': 14438,
  'imglink': 'https://spoonacular.com/recipeImages/14438-312x231.jpg',
  'missedCount': 4,
  'missedIngs': ['Endive and Asian Pear Salad',
   '1/4 teaspoon coarse kosher salt',
   '1/2 teaspoon coarsely ground black pepper',
   '2 tablespoons (1/4 stick) unsalted butter'],
  'name': 'Cabrales Cheese Souffles with Endive and Asian Pear Salad',
  'price': 41.9,
  'response': <Response [200]>,
  'serving': 6,
  'subIngs': {'ground pepper': {'hot pepper sauce': ['3/4 teaspoon cayenne pepper plus 1 teaspoon vinegar']},
   'unsalted butter': {'butter (salted)': ['1 cup margarine ',
     ' 1 cup shortening plus 1/2 teaspoon salt ',
     ' 7/8 cup vegetable oil plus 1/2 teaspoon salt ',
     ' 7/8 cup lard plus 1/2 teaspoon salt'],
    'butter (unsalted)': ['1 cup sh

In [5]:
r = allr.recser.iloc[0]

In [6]:
dbrfile = open('respr.pkl', 'rb') 
r.response =  dill.load(dbrfile)
r.setVars()

In [7]:
r.to_json()

{'attrs': {'vegetarian': False,
  'vegan': False,
  'glutenFree': False,
  'dairyFree': False},
 'cuisines': array(['Indian', 'Asian'], dtype='<U6'),
 'id': 14438,
 'imglink': 'https://spoonacular.com/recipeImages/14438-312x231.jpg',
 'missedCount': 4,
 'missedIngs': ['Endive and Asian Pear Salad',
  '1/4 teaspoon coarse kosher salt',
  '1/2 teaspoon coarsely ground black pepper',
  '2 tablespoons (1/4 stick) unsalted butter'],
 'name': 'Cabrales Cheese Souffles with Endive and Asian Pear Salad',
 'price': 41.9,
 'response': <Response [200]>,
 'serving': 6,
 'subIngs': {'ground pepper': {'hot pepper sauce': ['3/4 teaspoon cayenne pepper plus 1 teaspoon vinegar']},
  'unsalted butter': {'butter (salted)': ['1 cup margarine ',
    ' 1 cup shortening plus 1/2 teaspoon salt ',
    ' 7/8 cup vegetable oil plus 1/2 teaspoon salt ',
    ' 7/8 cup lard plus 1/2 teaspoon salt'],
   'butter (unsalted)': ['1 cup shortening ',
    ' 7/8 cup vegetable oil ',
    ' 7/8 cup lard']}},
 'urls': {'sourc